# Project 1
## The Net Wars: Attack of the Bots
### Advanced Machine Learning - MECD (2022/2023)

### Autores

- Duarte Meneses - 2019216949
- Patricia Costa - 2019213995

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import tensorflow as tf
from sklearn.metrics import classification_report

# Just to make plots look better
plt.rcParams["figure.figsize"] = (10,5)
plt.rcParams['axes.grid'] = True
plt.style.use('fivethirtyeight')
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['lines.linewidth'] = 3

|Class  |Label |
| ----- | ---- | 
|Normal	|0     |
|Dos	|1     |
|R2L	|2     |
|U2R	|3     |
|Probe	|4     |

In [2]:
PATH_TO_DATA = 'dataset/'

test_students = pd.read_csv(PATH_TO_DATA + 'test_students.csv')
SampleID = test_students['SampleID']

train_students = pd.read_csv(PATH_TO_DATA + 'train_students.csv')


In [3]:
test_students = test_students.drop('SampleID', axis=1)

### Check Nulls and DataType

In [4]:
print(test_students.info())
print(train_students.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44556 entries, 0 to 44555
Data columns (total 41 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   duration                     44556 non-null  int64  
 1   protocol_type                44556 non-null  object 
 2   service                      44556 non-null  object 
 3   flag                         44556 non-null  object 
 4   src_bytes                    44556 non-null  int64  
 5   dst_bytes                    44556 non-null  int64  
 6   land                         44556 non-null  int64  
 7   wrong_fragment               44556 non-null  int64  
 8   urgent                       44556 non-null  int64  
 9   hot                          44556 non-null  int64  
 10  num_failed_logins            44556 non-null  int64  
 11  logged_in                    44556 non-null  int64  
 12  num_compromised              44556 non-null  int64  
 13  root_shell      

In [5]:
train_students.replace('normal', 0, inplace=True)
train_students.replace('Dos', 1, inplace=True)
train_students.replace('R2L', 2, inplace=True)
train_students.replace('U2R', 3, inplace=True)
train_students.replace('Probe', 4, inplace=True)


In [6]:
train_students['attack_type'].value_counts(normalize = True)

0    0.518964
1    0.358846
4    0.095545
2    0.025808
3    0.000837
Name: attack_type, dtype: float64

### Categorical

In [7]:
#Service não contêm o mesmo núemero de valores
train_students = train_students.drop('service', axis=1)
test_students = test_students.drop('service', axis=1)

train_students = pd.get_dummies(train_students, columns=['protocol_type'], drop_first=True)
#train_students = train_students.drop('service', axis=1)
#train_students = pd.get_dummies(train_students, columns=['service'], drop_first=True)
train_students= pd.get_dummies(train_students, columns=['flag'], drop_first=True)

test_students = pd.get_dummies(test_students, columns=['protocol_type'], drop_first=True)
#test_students = test_students.drop('service', axis=1)
#test_students = pd.get_dummies(test_students, columns=['service'], drop_first=True)
test_students= pd.get_dummies(test_students, columns=['flag'], drop_first=True)


In [8]:
y = train_students['attack_type']
X = train_students.drop(columns=['attack_type'], axis=1)

### Primeira tentativa de ANN com SMOTE

In [9]:
oversample = SMOTE()
over_X, over_y = oversample.fit_resample(X, y)
over_X_train, over_X_test, over_y_train, over_y_test = train_test_split(over_X, over_y, test_size=0.2, stratify=over_y, random_state=42)

classifier = Sequential()

classifier.add(Dense(units=32, input_dim=50, kernel_initializer='uniform', activation='relu'))
classifier.add(Dense(units=16, kernel_initializer='uniform', activation='relu'))
 
classifier.add(Dense(units=5, kernel_initializer='uniform', activation='softmax'))
 
classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
 
survivalANN_Model=classifier.fit(over_X_train, over_y_train, batch_size=10 , epochs=10, verbose=1)


Epoch 1/10
21581/21581 [==============================] - 19s 852us/step - loss: 0.6163 - accuracy: 0.8607
Epoch 2/10
21581/21581 [==============================] - 19s 900us/step - loss: 0.3420 - accuracy: 0.9062
Epoch 3/10
21581/21581 [==============================] - 25s 1ms/step - loss: 0.2860 - accuracy: 0.9189
Epoch 4/10
21581/21581 [==============================] - 27s 1ms/step - loss: 0.2393 - accuracy: 0.9280
Epoch 5/10
21581/21581 [==============================] - 26s 1ms/step - loss: 0.2296 - accuracy: 0.9291
Epoch 6/10
21581/21581 [==============================] - 27s 1ms/step - loss: 0.3027 - accuracy: 0.9313
Epoch 7/10
21581/21581 [==============================] - 26s 1ms/step - loss: 5.1025 - accuracy: 0.9315
Epoch 8/10
21581/21581 [==============================] - 27s 1ms/step - loss: 0.2322 - accuracy: 0.9285
Epoch 9/10
21581/21581 [==============================] - 26s 1ms/step - loss: 0.2645 - accuracy: 0.9313
Epoch 10/10
21581/21581 [==========================

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [11]:
classifier.evaluate(X_test, y_test)
preds = classifier.predict(test_students)
predict_test = np.argmax(preds, axis=1)

1393/1393 [==============================] - 1s 1ms/step


In [ ]:
submission = pd.DataFrame({'SampleID': SampleID, 'Class': predict_test})
print(submission)
submission.to_csv('submission_ann.csv', index=False)

### Escolher melhores parâmetros

In [84]:
def FunctionFindBestParams(X_train, y_train):
    
    TrialNumber=0
    batch_size_list=[5, 10, 15, 20, 32]
    epoch_list=[5, 10, 15, 20, 32]
    
    import pandas as pd
    SearchResultsData=pd.DataFrame(columns=['TrialNumber', 'Batch', 'Epoch', 'Accuracy'])
    
    for batch_size_trial in batch_size_list:
        for epochs_trial in epoch_list:
            TrialNumber+=1
            
            classifier = Sequential()
            classifier.add(Dense(units=32, input_dim=50, kernel_initializer='uniform', activation='relu'))
            classifier.add(Dense(units=16, kernel_initializer='uniform', activation='relu'))
            classifier.add(Dense(units=5, kernel_initializer='uniform', activation='softmax'))
            classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
 
            
            survivalANN_Model=classifier.fit(X_train,y_train, batch_size=batch_size_trial , epochs=epochs_trial, verbose=1)

            Accuracy = survivalANN_Model.history['accuracy'][-1]
    
            print(" ")
            print(TrialNumber, 'Parameters:','batch_size:', batch_size_trial,'-', 'epochs:',epochs_trial, 'Accuracy:', Accuracy)
            
            print("---------------------------------------------")
            df = pd.DataFrame({'TrialNumber': [TrialNumber], 
                                'Batch': [batch_size_trial],
                                'Epoch': [epochs_trial],
                                'Accuracy': [Accuracy]})
            SearchResultsData = pd.concat([SearchResultsData, df], ignore_index=True)
    return(SearchResultsData)
 
###############################################
 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

ResultsData=FunctionFindBestParams(X_train, y_train)

Epoch 1/5
16634/16634 [==============================] - 21s 1ms/step - loss: 103.4161 - accuracy: 0.9102
Epoch 2/5
16634/16634 [==============================] - 43s 3ms/step - loss: 3.5716 - accuracy: 0.9380
Epoch 3/5
16634/16634 [==============================] - 22s 1ms/step - loss: 0.5725 - accuracy: 0.9428
Epoch 4/5
16634/16634 [==============================] - 43s 3ms/step - loss: 10.2281 - accuracy: 0.9490
Epoch 5/5
16634/16634 [==============================] - 26s 2ms/step - loss: 5.6808 - accuracy: 0.9504
 
1 Parameters: batch_size: 5 - epochs: 5 Accuracy: 0.9503535032272339
---------------------------------------------
Epoch 1/10
16634/16634 [==============================] - 35s 2ms/step - loss: 37.3429 - accuracy: 0.9204
Epoch 2/10
16634/16634 [==============================] - 28s 2ms/step - loss: 86.0273 - accuracy: 0.9420
Epoch 3/10
16634/16634 [==============================] - 26s 2ms/step - loss: 0.7632 - accuracy: 0.9463
Epoch 4/10
16634/16634 [===================

In [85]:
print(ResultsData)

   TrialNumber Batch Epoch  Accuracy
0            1     5     5  0.950354
1            2     5    10    0.9555
2            3     5    15  0.957977
3            4     5    20  0.958963
4            5     5    32  0.959432
5            6    10     5  0.952097
6            7    10    10   0.96376
7            8    10    15   0.95847
8            9    10    20  0.957243
9           10    10    32  0.961644
10          11    15     5  0.956594
11          12    15    10  0.952181
12          13    15    15  0.942406
13          14    15    20  0.962017
14          15    15    32  0.957604
15          16    20     5  0.950618
16          17    20    10  0.957892
17          18    20    15  0.956438
18          19    20    20  0.900755
19          20    20    32  0.951147
20          21    32     5  0.957063
21          22    32    10  0.955271
22          23    32    15  0.942959
23          24    32    20  0.969327
24          25    32    32  0.917733


In [ ]:
maxi = ResultsData[ResultsData.Accuracy == ResultsData.Accuracy.max()]
arg = maxi.iloc[0]

#Maxi = 32, 20

### Weights instead of SMOTE 

In [12]:
count = y.value_counts()

tam = len(y)
count/tam

0    0.518964
1    0.358846
4    0.095545
2    0.025808
3    0.000837
Name: attack_type, dtype: float64

In [13]:
class_weight1 = {0: (count[3]/tam), 1: (count[2]/tam), 2: (count[1]/tam), 3: (count[0]/tam), 4: (count[4]/tam)}

class_weight1

{0: 0.0008368522811438905,
 1: 0.0258077548311386,
 2: 0.35884610575119513,
 3: 0.5189638422100595,
 4: 0.0955454449264628}

In [14]:
class_weight2 = {0: 1-(count[0]/tam), 1: 1-(count[1]/tam), 2: 1-(count[2]/tam), 3: 1-(count[3]/tam), 4: 1-(count[4]/tam)}

class_weight2

{0: 0.48103615778994047,
 1: 0.6411538942488049,
 2: 0.9741922451688614,
 3: 0.9991631477188561,
 4: 0.9044545550735372}

### ANN with best parameters

In [15]:
'''
oversample = SMOTE()
over_X, over_y = oversample.fit_resample(X, y)
over_X_train, over_X_test, over_y_train, over_y_test = train_test_split(over_X, over_y, test_size=0.2, stratify=over_y, random_state=42)
'''


classifier = Sequential()

classifier.add(Dense(units=32, input_dim=50, kernel_initializer='uniform', activation='relu'))
classifier.add(Dense(units=16, kernel_initializer='uniform', activation='relu'))
 

classifier.add(Dense(units=5, kernel_initializer='uniform', activation='softmax'))

classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
 
survivalANN_Model=classifier.fit(X_train, y_train, batch_size=32 , epochs=20, verbose=1)
#survivalANN_Model=classifier.fit(X_train, y_train, batch_size=arg.Batch , epochs=arg.Epoch, verbose=1, class_weight=class_weight)

Epoch 1/20
2599/2599 [==============================] - 4s 1ms/step - loss: 49.0306 - accuracy: 0.8799
Epoch 2/20
2599/2599 [==============================] - 3s 1ms/step - loss: 14.0479 - accuracy: 0.9053
Epoch 3/20
2599/2599 [==============================] - 3s 1ms/step - loss: 0.2572 - accuracy: 0.9358
Epoch 4/20
2599/2599 [==============================] - 3s 1ms/step - loss: 0.2157 - accuracy: 0.9420
Epoch 5/20
2599/2599 [==============================] - 3s 1ms/step - loss: 0.1931 - accuracy: 0.9472
Epoch 6/20
2599/2599 [==============================] - 3s 1ms/step - loss: 0.1928 - accuracy: 0.9469
Epoch 7/20
2599/2599 [==============================] - 3s 1ms/step - loss: 0.1759 - accuracy: 0.9496
Epoch 8/20
2599/2599 [==============================] - 3s 1ms/step - loss: 0.7648 - accuracy: 0.9514
Epoch 9/20
2599/2599 [==============================] - 3s 1ms/step - loss: 0.1427 - accuracy: 0.9582
Epoch 10/20
2599/2599 [==============================] - 3s 1ms/step - loss: 0.1

In [16]:
classifier.evaluate(X_test, y_test)

pred = classifier.predict(X_test)
pred = np.argmax(pred, axis=1)

print(classification_report(y_test, pred, zero_division=0))
print('=========================================')


650/650 [==============================] - 1s 898us/step
              precision    recall  f1-score   support

           0       0.96      0.99      0.97     10791
           1       0.99      0.96      0.98      7461
           2       0.89      0.80      0.84       537
           3       0.00      0.00      0.00        17
           4       0.98      0.97      0.97      1987

    accuracy                           0.97     20793
   macro avg       0.76      0.74      0.75     20793
weighted avg       0.97      0.97      0.97     20793



### ANN with Weights 1

In [17]:
classifier = Sequential()

classifier.add(Dense(units=32, input_dim=50, kernel_initializer='uniform', activation='relu'))
classifier.add(Dense(units=16, kernel_initializer='uniform', activation='relu'))
 

classifier.add(Dense(units=5, kernel_initializer='uniform', activation='softmax'))

classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
 
survivalANN_Model=classifier.fit(X_train, y_train, batch_size=32 , epochs=20, verbose=1, class_weight = class_weight1)

Epoch 1/20
2599/2599 [==============================] - 4s 1ms/step - loss: 28.6129 - accuracy: 0.4875
Epoch 2/20
2599/2599 [==============================] - 3s 1ms/step - loss: 2.1418 - accuracy: 0.6298
Epoch 3/20
2599/2599 [==============================] - 3s 1ms/step - loss: 7.3966 - accuracy: 0.6151
Epoch 4/20
2599/2599 [==============================] - 4s 1ms/step - loss: 40.0591 - accuracy: 0.6820
Epoch 5/20
2599/2599 [==============================] - 3s 1ms/step - loss: 16.8136 - accuracy: 0.5404
Epoch 6/20
2599/2599 [==============================] - 4s 1ms/step - loss: 3.5333 - accuracy: 0.5668
Epoch 7/20
2599/2599 [==============================] - 4s 1ms/step - loss: 12.2307 - accuracy: 0.5550
Epoch 8/20
2599/2599 [==============================] - 4s 1ms/step - loss: 37.9693 - accuracy: 0.6698
Epoch 9/20
2599/2599 [==============================] - 4s 1ms/step - loss: 10.5366 - accuracy: 0.6949
Epoch 10/20
2599/2599 [==============================] - 4s 1ms/step - loss:

In [18]:
classifier.evaluate(X_test, y_test)

pred = classifier.predict(X_test)
pred = np.argmax(pred, axis=1)

print(classification_report(y_test, pred))
print('=========================================')

650/650 [==============================] - 1s 1ms/step
              precision    recall  f1-score   support

           0       1.00      0.45      0.62     10791
           1       0.88      0.95      0.91      7461
           2       0.15      0.96      0.25       537
           3       0.00      0.29      0.01        17
           4       0.64      0.98      0.77      1987

    accuracy                           0.69     20793
   macro avg       0.53      0.73      0.51     20793
weighted avg       0.90      0.69      0.73     20793



### ANN with Weights 2

In [19]:
classifier = Sequential()

classifier.add(Dense(units=32, input_dim=50, kernel_initializer='uniform', activation='relu'))
classifier.add(Dense(units=16, kernel_initializer='uniform', activation='relu'))
 

classifier.add(Dense(units=5, kernel_initializer='uniform', activation='softmax'))

classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
 
survivalANN_Model=classifier.fit(X_train, y_train, batch_size=32 , epochs=20, verbose=1, class_weight = class_weight2)

Epoch 1/20
2599/2599 [==============================] - 4s 1ms/step - loss: 16.7489 - accuracy: 0.8810
Epoch 2/20
2599/2599 [==============================] - 3s 1ms/step - loss: 16.7809 - accuracy: 0.9232
Epoch 3/20
2599/2599 [==============================] - 3s 1ms/step - loss: 0.1400 - accuracy: 0.9409
Epoch 4/20
2599/2599 [==============================] - 4s 1ms/step - loss: 7.4376 - accuracy: 0.9389
Epoch 5/20
2599/2599 [==============================] - 3s 1ms/step - loss: 0.2460 - accuracy: 0.9469
Epoch 6/20
2599/2599 [==============================] - 4s 1ms/step - loss: 0.1127 - accuracy: 0.9499
Epoch 7/20
2599/2599 [==============================] - 3s 1ms/step - loss: 0.7822 - accuracy: 0.9520
Epoch 8/20
2599/2599 [==============================] - 4s 1ms/step - loss: 0.0986 - accuracy: 0.9549
Epoch 9/20
2599/2599 [==============================] - 4s 1ms/step - loss: 2.1852 - accuracy: 0.9540
Epoch 10/20
2599/2599 [==============================] - 4s 1ms/step - loss: 0.1

In [20]:
classifier.evaluate(X_test, y_test)

pred = classifier.predict(X_test)
pred = np.argmax(pred, axis=1)

print(classification_report(y_test, pred))
print('=========================================')

650/650 [==============================] - 1s 863us/step
              precision    recall  f1-score   support

           0       0.94      0.98      0.96     10791
           1       0.99      0.95      0.97      7461
           2       0.78      0.81      0.79       537
           3       0.00      0.00      0.00        17
           4       0.97      0.92      0.95      1987

    accuracy                           0.96     20793
   macro avg       0.74      0.73      0.73     20793
weighted avg       0.96      0.96      0.96     20793



c:\Users\emanu\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\emanu\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\emanu\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
preds = classifier.predict(test_students)
predict_test = np.argmax(preds, axis=1)

1393/1393 [==============================] - 1s 1ms/step


In [ ]:
submission = pd.DataFrame({'SampleID': SampleID, 'Class': predict_test})
print(submission)
submission.to_csv('submission_ann_weights.csv', index=False)

       SampleID  Class
0             0      2
1             1      0
2             2      0
3             3      0
4             4      1
...         ...    ...
44551     44551      0
44552     44552      1
44553     44553      1
44554     44554      0
44555     44555      0

[44556 rows x 2 columns]
